In [2]:
!pip install pandas

import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df = pd.read_csv("/train_df.csv")

X = df.drop(columns=["Target"])
y = df["Target"]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_sets = [
    {"C": 0.1, "kernel": "linear"},
    {"C": 1, "kernel": "linear"},
    {"C": 1, "kernel": "rbf", "gamma": "scale"},
    {"C": 10, "kernel": "rbf", "gamma": "scale"},
    {"C": 10, "kernel": "rbf", "gamma": "auto"},
]

results = []

for params in param_sets:
    fold_metrics = []

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model = SVC(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="weighted", zero_division=0)
        recall = recall_score(y_test, y_pred, average="weighted", zero_division=0)
        f1 = f1_score(y_test, y_pred, average="weighted", zero_division=0)

        fold_metrics.append({"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1})

    avg_metrics = {
        "params": params,
        "accuracy": sum(f["accuracy"] for f in fold_metrics) / skf.get_n_splits(),
        "precision": sum(f["precision"] for f in fold_metrics) / skf.get_n_splits(),
        "recall": sum(f["recall"] for f in fold_metrics) / skf.get_n_splits(),
        "f1_score": sum(f["f1_score"] for f in fold_metrics) / skf.get_n_splits(),
    }
    results.append(avg_metrics)

results

[{'params': {'C': 0.1, 'kernel': 'linear'},
  'accuracy': 0.8633748501745793,
  'precision': 0.8646171326080582,
  'recall': 0.8633748501745793,
  'f1_score': 0.8581729828231314},
 {'params': {'C': 1, 'kernel': 'linear'},
  'accuracy': 0.8672499869717025,
  'precision': 0.8677294651144404,
  'recall': 0.8672499869717025,
  'f1_score': 0.8628592314533948},
 {'params': {'C': 1, 'kernel': 'rbf', 'gamma': 'scale'},
  'accuracy': 0.8543332117358904,
  'precision': 0.8577705121560213,
  'recall': 0.8543332117358904,
  'f1_score': 0.8468018787833834},
 {'params': {'C': 10, 'kernel': 'rbf', 'gamma': 'scale'},
  'accuracy': 0.8423794882484756,
  'precision': 0.839606558719429,
  'recall': 0.8423794882484756,
  'f1_score': 0.8385847606111888},
 {'params': {'C': 10, 'kernel': 'rbf', 'gamma': 'auto'},
  'accuracy': 0.8423794882484756,
  'precision': 0.839606558719429,
  'recall': 0.8423794882484756,
  'f1_score': 0.8385847606111888}]